In [1]:
import pandas as pd
import ast

In [2]:
mal1 = pd.read_csv('/scratch/cm/malayalam_train.tsv', sep='\t')
mal2 = pd.read_csv('/scratch/cm/malayalam_dev.tsv', sep='\t')
mal3 = pd.read_csv('/scratch/cm/malayalam_test_results - malayalam_test_results.tsv', sep='\t')

In [3]:
df1 = mal1.append(mal2).append(mal3)

In [4]:
df1 = df1[df1.category!='not-malayalam']
df1 = df1[df1.category!='not-malayalam ']
df1 = df1.drop(['id', 'category'], axis = 1)
df1['lang'] = 4

In [5]:
tam1 = pd.read_csv('/scratch/cm/tamil_train.tsv', sep='\t')
tam2 = pd.read_csv('/scratch/cm/tamil_dev.tsv', sep='\t')
tam3 = pd.read_csv('/scratch/cm/tamil_test_answer - tamil_test_answer.tsv', sep='\t')

In [6]:
df2 = tam1.append(tam2).append(tam3)

In [7]:
df2 = df2[df2.category!='not-Tamil']
df2 = df2[df2.category!='not-Tamil ']
df2 = df2.drop(['id', 'category'], axis = 1)
df2['lang'] = 3

In [8]:
df3 = pd.read_json('/scratch/cm/hinglishNorm.json')

In [9]:
df3.tags = df3.tags.apply(ast.literal_eval)

In [10]:
def lgood(ls):
    sm = sum([i == 'Looks Good' for i in ls])
    return 1 if sm/len(ls) < 0.6 and len(ls) >= 4 else 0
df3['codemix'] = df3.tags.apply(lgood)
df3 = df3[df3['codemix'] == 1]

In [11]:
df3

,id,inputText,tags,normalizedText,codemix
1,2,tarika hai bolne ka,"[Hindi, Hindi, Hindi, Hindi]",tarika hai bolne ka,1
7,8,yaar jo bhi ho na mast ho tum,"[Hindi, Hindi, Hindi, Hindi, Hindi, Hindi, Hin...",yaar jo bhee ho na mast ho tum,1
8,9,or jo ahae dekhlenge,"[Hindi, Hindi, Hindi, Hindi]",aur jo he dekhalenge,1
11,12,koi na baat kar lo,"[Hindi, Hindi, Hindi, Hindi, Hindi]",koee na baat kar lo,1
12,13,ab gate k through recruitment hoga,"[Hindi, Acronym, Unrecognizable or other langu...",ab graduate aptitude test in engineering ke th...,1
...,...,...,...,...,...
13482,13483,kisi se mi phone maang le,"[Hindi, Hindi, Hindi, Looks Good, Hindi, Hindi]",kisee se bhee phone maang le,1
13483,13484,koi nhi tu v kr lena post graduation,"[Hindi, Hindi, Hindi, Hindi, Hindi, Hindi, Loo...",koee nahin too bhee kar lena post graduation,1
13485,13486,th kya kr liya,"[Hindi, Hindi, Hindi, Hindi]",tha kya kar liya,1
13488,13489,ye kya hota h,"[Hindi, Hindi, Hindi, Hindi]",ye kya hota hai,1


In [12]:
df3['text'] = df3['normalizedText']
df3['lang'] = 2
df3 = df3.drop(['tags', 'normalizedText', 'codemix', 'inputText', 'id'], axis = 1)


In [13]:
with open('/scratch/cm/all_wiki_sents.txt', 'r') as f:
    st =  f.read()
    sents = st.split('\n')
    df4 = pd.DataFrame(sents[:-1])

In [14]:
df4['text'] = df4[0]
df4['lang'] = 1
df4 = df4.drop([0], axis = 1)


In [15]:
df1.text = df1.text.str.lower()
df2.text = df2.text.str.lower()
df3.text = df3.text.str.lower()
df4.text = df4.text.str.lower()

In [16]:
mn = 5691
df1 = df1.sample(mn)
df2 = df2.sample(mn)
df3 = df3.sample(mn)
df4 = df4.sample(mn)

In [17]:
df1.reset_index(drop=True, inplace=True)
df2.reset_index(drop=True, inplace=True)
df3.reset_index(drop=True, inplace=True)
df4.reset_index(drop=True, inplace=True)

In [18]:
df = df1.append(df2).append(df3).append(df4)

In [19]:
df = df.sample(len(df))
df.reset_index(drop=True, inplace=True)

In [20]:
from sklearn.model_selection import train_test_split as tts
train, valid = tts(df, test_size = 0.2, random_state = 42, stratify=df.lang)

In [21]:
valid

,text,lang
12175,it also contains punishment for transgression,1
14738,evidence for lilith being a goddess rather tha...,1
3654,video va unlyk panravangellam avarooda hardwo...,3
22287,ryan died in toronto at age 89,1
7882,nte ponno oru rakshayum illatta makinggg........,4
...,...,...
14909,vere level ... onnum parayan ella....,4
10606,a miami,1
12798,aapako twitter hack karana aata hai kya,2
3310,mammooka fans vedipooram .. vanne .. adi like...,4


In [22]:
train.to_csv('train.csv')
valid.to_csv('valid.csv')

In [23]:

def make_vocab(count, min_freq=3, max_vocab=60000, special_toks=None):
    "Create a vocab of `max_vocab` size from `Counter` `count` with items present more than `min_freq`"
    vocab = [o for o,c in count.most_common(max_vocab) if c >= min_freq]
    special_toks = ifnone(special_toks, 'n')
    for o in reversed(special_toks): #Make sure all special tokens are in the vocab
        if o in vocab: vocab.remove(o)
        vocab.insert(0, o)
    vocab = vocab[:max_vocab]
    return vocab + [f'xxfake' for i in range(0, 8-len(vocab)%8)]
def ifnone(a,b):
    "`a` if `a` is not None, otherwise `b`."
    return b if a is None else a

In [24]:
vocab = list()
for i in list(df.text.values):
    tmp = []
    for j in i.split(' '):
        if len(j)> 0:
            tmp.append(j)
    vocab.append(tmp)

In [25]:
import pickle
with open('vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)

In [26]:
vocab[3]

['kattaa', 'waiting', 'by', 'a', 'lalettan', 'fan#ikkaa', 'vere', 'level']

In [30]:
train['is_valid'] = 0
valid['is_valid'] = 1
train = train.append(valid)
train.to_csv('total.csv')

<ipython-input-30-a3f33f21fc13>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['is_valid'] = 0
<ipython-input-30-a3f33f21fc13>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['is_valid'] = 1


In [31]:
train

,text,lang,is_valid
19306,1.01 prithviraj anenn manasilayavar like adik,4,0
13031,rajani sir gr8 man agree hit like,3,0
3135,it also features motion blur unlike its predec...,1,0
1767,petta vs viswasam petta like viswasam comment,3,0
7086,but even then yaar real identity toh connect k...,2,0
...,...,...,...
14909,vere level ... onnum parayan ella....,4,1
10606,a miami,1,1
12798,aapako twitter hack karana aata hai kya,2,1
3310,mammooka fans vedipooram .. vanne .. adi like...,4,1
